Leitura dos dados no documento excel `Acoustic_Extinguisher_Fire_Dataset.xlsx`.

In [18]:
import numpy as np
import pandas as pd

df = pd.read_excel('./Acoustic_Extinguisher_Fire_Dataset/Acoustic_Extinguisher_Fire_Dataset.xlsx')
df.head()

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
0,1,gasoline,10,96,0.0,75,0
1,1,gasoline,10,96,0.0,72,1
2,1,gasoline,10,96,2.6,70,1
3,1,gasoline,10,96,3.2,68,1
4,1,gasoline,10,109,4.5,67,1


In [19]:
desvp = df['AIRFLOW'].std()
desvp

4.736168980755709

Detecção e remoção dos outliers no atributo **`AIRFLOW`** pela distribuição dos dados.

In [20]:
desvp = df['AIRFLOW'].std()
media = df['AIRFLOW'].mean()

print("media = %.2f, desvio padrao = %.2f" % (media, desvp))

# apenas outliers segundo std
df_out_airflow = df[(df['AIRFLOW'] < media-(desvp*2)) | (df['AIRFLOW'] > media+(desvp*2))]
print("Outliers:", len(df_out_airflow))

# apenas inliers segundo std
df_in_airflow = df[(df['AIRFLOW'] >= media-(desvp*2)) & (df['AIRFLOW'] <= media+(desvp*2))]
print("Inliers:", len(df_in_airflow))

#removendo outliers
df = df_in_airflow

media = 6.98, desvio padrao = 4.74
Outliers: 34
Inliers: 17408


Detecção e remoção dos outliers no atributo **`DESIBEL`** pela distribuição dos dados.

In [21]:
desvp = df['DESIBEL'].std()
media = df['DESIBEL'].mean()

print("media = %.2f, desvio padrao = %.2f" % (media, desvp))

# apenas outliers segundo std
df_out_desibel = df[(df['DESIBEL'] < media-(desvp*2)) | (df['DESIBEL'] > media+(desvp*2))]
print("Outliers:", len(df_out_desibel))

# apenas inliers segundo std
df_in_desibel = df[(df['DESIBEL'] >= media-(desvp*2)) & (df['DESIBEL'] <= media+(desvp*2))]
print("Inliers:", len(df_in_desibel))

#removendo outliers
df = df_in_desibel

media = 96.35, desvio padrao = 8.15
Outliers: 595
Inliers: 16813


Detecção e remoção dos outliers no atributo **`FREQUENCY`** pela distribuição dos dados.

In [22]:
desvp = df['FREQUENCY'].std()
media = df['FREQUENCY'].mean()

print("media = %.2f, desvio padrao = %.2f" % (media, desvp))

# apenas outliers segundo std
df_out_freq = df[(df['FREQUENCY'] < media-(desvp*2)) | (df['FREQUENCY'] > media+(desvp*2))]
print("Outliers:", len(df_out_freq))

# apenas inliers segundo std
df_in_freq = df[(df['FREQUENCY'] >= media-(desvp*2)) & (df['FREQUENCY'] <= media+(desvp*2))]
print("Inliers:", len(df_in_freq))

#removendo outliers
df = df_in_freq

media = 32.63, desvio padrao = 20.55
Outliers: 323
Inliers: 16490


Detecção e remoção dos outliers no atributo **`DISTANCE`** pela distribuição dos dados.

In [23]:
desvp = df['DISTANCE'].std()
media = df['DISTANCE'].mean()

print("media = %.2f, desvio padrao = %.2f" % (media, desvp))

# apenas outliers segundo std
df_out_dist = df[(df['DISTANCE'] < media-(desvp*2)) | (df['DISTANCE'] > media+(desvp*2))]
print("Outliers:", len(df_out_dist))

# apenas inliers segundo std
df_in_dist = df[(df['DISTANCE'] >= media-(desvp*2)) & (df['DISTANCE'] <= media+(desvp*2))]
print("Inliers:", len(df_in_dist))

#removendo outliers
df = df_in_dist

media = 100.06, desvio padrao = 54.54
Outliers: 0
Inliers: 16490


Verificação de dados faltando em todas as colunas.

In [24]:
df.isna().sum()

SIZE         0
FUEL         0
DISTANCE     0
DESIBEL      0
AIRFLOW      0
FREQUENCY    0
STATUS       0
dtype: int64

Verficação de dados nulos em todas as colunas

In [25]:
df.isnull().sum()

SIZE         0
FUEL         0
DISTANCE     0
DESIBEL      0
AIRFLOW      0
FREQUENCY    0
STATUS       0
dtype: int64

Dados após o tratamento de outliers.

In [16]:
df

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
1,1,gasoline,10,96,0.0,72,1
2,1,gasoline,10,96,2.6,70,1
3,1,gasoline,10,96,3.2,68,1
4,1,gasoline,10,109,4.5,67,1
5,1,gasoline,10,109,7.8,66,1
...,...,...,...,...,...,...,...
17434,7,lpg,190,91,1.9,8,0
17435,7,lpg,190,91,1.6,7,0
17436,7,lpg,190,87,2.5,6,0
17437,7,lpg,190,86,2.2,5,0


Convertemos os atributos categóricos **FUEL** em numéricos usando a técnica one-hot encoding através do método `get_dummies` no `pandas`.

Como resultado, os quatro tipos de combustíveis presentes (`gasoline`, `kerosene`, `lpg` e `thinner`) foram separados em 4 atributos com valores 0 ou 1.

In [17]:
dummy_vars = pd.get_dummies(df['FUEL'])
df_dummies = pd.concat([dummy_vars, df.drop('FUEL', axis=1)], axis = 1, sort=False)
df_dummies

,gasoline,kerosene,lpg,thinner,SIZE,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
1,1,0,0,0,1,10,96,0.0,72,1
2,1,0,0,0,1,10,96,2.6,70,1
3,1,0,0,0,1,10,96,3.2,68,1
4,1,0,0,0,1,10,109,4.5,67,1
5,1,0,0,0,1,10,109,7.8,66,1
...,...,...,...,...,...,...,...,...,...,...
17434,0,0,1,0,7,190,91,1.9,8,0
17435,0,0,1,0,7,190,91,1.6,7,0
17436,0,0,1,0,7,190,87,2.5,6,0
17437,0,0,1,0,7,190,86,2.2,5,0
